In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import os

In [2]:
seed = 0
np.random.seed(seed)
torch.manual_seed(seed)

In [3]:
datapath = os.path.join("..", "data", "pima-indians-diabetes.csv")
data = np.loadtxt(datapath, delimiter = ',', dtype = np.float32)

x_train = data[:700, 0:8]
y_train = data[:700, 8].reshape(-1, 1)
x_test = data[700:, 0:8]
y_test = data[700:, 8].reshape(-1, 1)

In [4]:
x_train = torch.tensor(x_train)
y_train = torch.tensor(y_train)
x_test = torch.tensor(x_test)
y_test = torch.tensor(y_test)

In [5]:
class SimpleModel(nn.Module):
    def __init__ (self):
        super(SimpleModel, self).__init__()
        self.layer1 = nn.Linear(8, 12)
        self.layer2 = nn.Linear(12, 8)
        self.layer3 = nn.Linear(8, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.sigmoid(self.layer3(x))
        return x

In [6]:
model = SimpleModel()

In [7]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.01)

In [8]:
epochs = 300
batch_size = 64

In [9]:
for epoch in range(epochs):
    model.train()
    permutation = torch.randperm(x_train.size(0))
    for i in range(0, x_train.size(0), batch_size):
        indices = permutation[i:i + batch_size]
        batch_x, batch_y = x_train[indices], y_train[indices]

        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch + 1} / {epochs}], Loss: {loss.item():.4f}")

Epoch [10 / 300], Loss: 0.6541
Epoch [20 / 300], Loss: 0.5165
Epoch [30 / 300], Loss: 0.5428
Epoch [40 / 300], Loss: 0.5966
Epoch [50 / 300], Loss: 0.5049
Epoch [60 / 300], Loss: 0.5142
Epoch [70 / 300], Loss: 0.4713
Epoch [80 / 300], Loss: 0.4755
Epoch [90 / 300], Loss: 0.5593
Epoch [100 / 300], Loss: 0.4726
Epoch [110 / 300], Loss: 0.5234
Epoch [120 / 300], Loss: 0.5502
Epoch [130 / 300], Loss: 0.3813
Epoch [140 / 300], Loss: 0.3159
Epoch [150 / 300], Loss: 0.4189
Epoch [160 / 300], Loss: 0.5462
Epoch [170 / 300], Loss: 0.4334
Epoch [180 / 300], Loss: 0.4230
Epoch [190 / 300], Loss: 0.6368
Epoch [200 / 300], Loss: 0.4505
Epoch [210 / 300], Loss: 0.3883
Epoch [220 / 300], Loss: 0.3338
Epoch [230 / 300], Loss: 0.4233
Epoch [240 / 300], Loss: 0.5101
Epoch [250 / 300], Loss: 0.4996
Epoch [260 / 300], Loss: 0.4348
Epoch [270 / 300], Loss: 0.5688
Epoch [280 / 300], Loss: 0.3808
Epoch [290 / 300], Loss: 0.4123
Epoch [300 / 300], Loss: 0.4343


In [10]:
model.eval()
with torch.no_grad():
    outputs = model(x_test)
    predicted = (outputs > 0.5).float()
    accuracy = (predicted.eq(y_test).sum() / y_test.size(0)).item() * 100
    print(f'Accuracy: {accuracy: .2f}%')

Accuracy:  80.88%
